In [1]:
import tensorflow as tf
from sklearn.datasets import fetch_mldata
import numpy as np

In [2]:
n_inputs = 28*28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

In [3]:
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

In [4]:
"""def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2 / np.sqrt(n_inputs)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        W = tf.Variable(init, name="weights")
        b = tf.Variable(tf.zeros([n_neurons]), name="biases")
        z = tf.matmul(X, W) + b
        if activation == "relu":
            return tf.nn.relu(z)
        else:
            return z"""

'def neuron_layer(X, n_neurons, name, activation=None):\n    with tf.name_scope(name):\n        n_inputs = int(X.get_shape()[1])\n        stddev = 2 / np.sqrt(n_inputs)\n        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)\n        W = tf.Variable(init, name="weights")\n        b = tf.Variable(tf.zeros([n_neurons]), name="biases")\n        z = tf.matmul(X, W) + b\n        if activation == "relu":\n            return tf.nn.relu(z)\n        else:\n            return z'

In [5]:
"""with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, "hidden1", activation="relu")
    hidden2 = neuron_layer(hidden1, n_hidden2, "hidden2", activation="relu")
    logits = neuron_layer(hidden2, n_outputs, "outputs")"""

'with tf.name_scope("dnn"):\n    hidden1 = neuron_layer(X, n_hidden1, "hidden1", activation="relu")\n    hidden2 = neuron_layer(hidden1, n_hidden2, "hidden2", activation="relu")\n    logits = neuron_layer(hidden2, n_outputs, "outputs")'

In [16]:
from datetime import datetime
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs_plainTF"
logdir = "{}/run-{}/".format(root_logdir, now)

In [7]:
from tensorflow.contrib.layers import fully_connected

In [8]:
with tf.name_scope("dnn"):
    hidden1 = fully_connected(X, n_hidden1, scope='hidden1')
    hidden2 = fully_connected(hidden1, n_hidden2, scope='hidden2')
    logits = fully_connected(hidden2, n_outputs, scope='outputs', activation_fn=None)

In [9]:
with tf.name_scope("loss"):
    xentrophy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentrophy, name="loss")

In [10]:
learning_rate = 0.02

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [11]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [17]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())
loss_summary = tf.summary.scalar('loss', loss)
acc_train_summary = tf.summary.scalar('Train_accuracy', accuracy)
acc_test_summary = tf.summary.scalar('Test_accuracy', accuracy)

In [18]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("/tmp/data")

Extracting /tmp/data\train-images-idx3-ubyte.gz
Extracting /tmp/data\train-labels-idx1-ubyte.gz
Extracting /tmp/data\t10k-images-idx3-ubyte.gz
Extracting /tmp/data\t10k-labels-idx1-ubyte.gz


In [19]:
n_epochs = 40
batch_size = 50

In [20]:
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            
        summary_acc_train = acc_train_summary.eval(feed_dict={X: X_batch, y: y_batch})
        summary_acc_test = acc_test_summary.eval(feed_dict={X: mnist.test.images, y: mnist.test.labels})
        summary_loss = loss_summary.eval(feed_dict={X: X_batch, y: y_batch})
        
        file_writer.add_summary(summary_acc_train, epoch)
        file_writer.add_summary(summary_acc_test, epoch)
        file_writer.add_summary(summary_loss, epoch)
        
        
    save_path = saver.save(sess, "/plainTF/my_final_model")